In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rabieelkharoua/alzheimers-disease-dataset")

print("Path to dataset files:", path)

# https://www.kaggle.com/code/adhamtarek147/alzheimer-s-disease-prediction
# Eksempel av løsning på datasettet

In [ ]:
import pandas as pd
data = pd.read_csv(path + "/alzheimers_disease_data.csv")
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
#som beskrevet i datasett sin beskrivelse  er doctorInCharge bare XXXconfid og dermed ikke relevant
data_cleaned = data.drop(columns=["DoctorInCharge"], axis=1)
#vi trenger heller ikkje PatientID
data_cleaned = data_cleaned.drop(columns=["PatientID"], axis=1)
data_cleaned.info()

In [ ]:
# Siden dette bare en enkel nettside så blir det vanskielig å få tilgang til medisisnke verdiser som CholesterolHDL og lignende.
# Så dermed fjerner vi disse kolonnene siden vi ikkje kan forvente brukeren å kunne disse verdiene.
data_cleaned = data_cleaned.drop(columns=["CholesterolTotal","CholesterolLDL","CholesterolHDL","CholesterolTriglycerides","MMSE","FunctionalAssessment","ADL"], axis=1)
data_cleaned.info()

In [ ]:
#

In [ ]:
for column in data_cleaned.columns:
    unique_values = data_cleaned[column].unique()
    print(f"Column '{column}' has {unique_values}.")

# vi ser her at et veldig stor foskjell mellom skaleringen til nokon av verdiene med, tall fra 0-3 og nokon fra 0-100
# vi må normalisere de

In [ ]:
from sklearn.preprocessing import StandardScaler

columns_to_normalize = ['Age','BMI', 'AlcoholConsumption', 'PhysicalActivity', 'DietQuality', 'SleepQuality', 'SystolicBP','DiastolicBP']

# skalere dataen
standard_scaler = StandardScaler()
data_cleaned[columns_to_normalize] = standard_scaler.fit_transform(data_cleaned[columns_to_normalize])
data_cleaned.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Lagd av Gemeni
# Encode the target variable since models require numerical input
le = LabelEncoder()
data_cleaned['Diagnosis'] = le.fit_transform(data_cleaned['Diagnosis'])

X = data_cleaned.drop(columns=["Diagnosis"], axis=1)
y = data_cleaned["Diagnosis"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


In [26]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

# vi tester flere modeler samtidig for å finne den beste
# vi tester da gjerne modeler som har en predict_proba metode siden alzheimers som kan være skadelig å gi konkrette spådommer om
# så vil bare gi en sannsynlighet for å ha sjukdommen og en anbefaling om å ta vidare tester

# Med å sjå på eksempel løsning på datasetet(https://www.kaggle.com/code/adhamtarek147/alzheimer-s-disease-prediction) så ser vi at DecisionTreeClassifier, RandomForestClassifier, XGBClassifier og CatBoostClassifier gir gode resultater
# Dermed så tester vi disse modellene her for å se hvilken av de er den beste på vårt modifisert datasett
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(eval_metric='mlogloss'),
    "CatBoost": CatBoostClassifier(verbose=0)
}

# Gitt av https://www.kaggle.com/code/adhamtarek147/alzheimer-s-disease-prediction
param_grids = {
    'Decision Tree': {'max_depth': [3, 5, 7, 12, None]},
    'Random Forest': {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7, 12, None]},
    'XGBoost': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1], 'max_depth': [3, 5, 7]},
    'CatBoost': {'iterations': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1]}
}

# Skrevet av Gemeni
for name, model in models.items():
    # Use 'f1_weighted' for multiclass classification scoring
    grid_search = GridSearchCV(model, param_grids[name], cv=5, scoring='f1_weighted')
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    # Inverse transform predictions to get original labels for the report
    y_test_labels = le.inverse_transform(y_test)
    y_pred_labels = le.inverse_transform(y_pred)

    report = classification_report(y_test_labels, y_pred_labels)
    print(f'{name} Classification Report:\n{report}\nBest Parameters: {grid_search.best_params_}\n')

Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.85      0.79       277
           1       0.63      0.46      0.54       153

    accuracy                           0.71       430
   macro avg       0.69      0.66      0.66       430
weighted avg       0.70      0.71      0.70       430

Best Parameters: {'max_depth': 3}

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.88      0.80       277
           1       0.67      0.44      0.53       153

    accuracy                           0.72       430
   macro avg       0.70      0.66      0.67       430
weighted avg       0.71      0.72      0.71       430

Best Parameters: {'max_depth': 12, 'n_estimators': 200}

XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.86      0.80       277
           1       0.64      0.46      0

In [25]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# Vi ser her at det var en stor forskjell i restultatene som vi fikk fra eksempel løsningen på datasettet og det vi fikk her
# Dermed så kan det være lurt å prøve de andre modellene også som er nevnt i eksempel løsningen på datasettet
# siden det kan hende at de er bedre egnet for dette modifiserte datasettet

models = {
    'Support Vector Machine': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Logistic Regression': LogisticRegression(),
}

# Gitt av https://www.kaggle.com/code/adhamtarek147/alzheimer-s-disease-prediction
param_grids = {
    'K-Nearest Neighbors': {'n_neighbors': [3, 5, 7]},
    'Logistic Regression': {'C': [0.1, 1, 10]},
    'Support Vector Machine': {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 'scale', 'auto']},
}

# Skrevet av Gemeni
for name, model in models.items():
    # Use 'f1_weighted' for multiclass classification scoring
    grid_search = GridSearchCV(model, param_grids[name], cv=5, scoring='f1_weighted')
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    # Inverse transform predictions to get original labels for the report
    y_test_labels = le.inverse_transform(y_test)
    y_pred_labels = le.inverse_transform(y_pred)

    report = classification_report(y_test_labels, y_pred_labels)
    print(f'{name} Classification Report:\n{report}\nBest Parameters: {grid_search.best_params_}\n')

Support Vector Machine Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.89      0.80       277
           1       0.67      0.41      0.50       153

    accuracy                           0.72       430
   macro avg       0.70      0.65      0.65       430
weighted avg       0.71      0.72      0.70       430

Best Parameters: {'C': 1, 'gamma': 'auto'}

K-Nearest Neighbors Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.77      0.71       277
           1       0.39      0.26      0.31       153

    accuracy                           0.59       430
   macro avg       0.52      0.52      0.51       430
weighted avg       0.56      0.59      0.57       430

Best Parameters: {'n_neighbors': 3}

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.85      0.80       277
           1       0.66 

In [ ]:
# Her såg vi at faktisk så var Logistic Regression den beste modellen for dette modifiserte datasettet
# med en vektet f1-score på 0.72
# Dermed så bruker vi denne modellen videre

